## α-β-Pruning

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
%run ./nmm-game.ipynb
%run ./nmm-heuristic.ipynb
%run ./nmm-symmetry.ipynb

Die Hilfsfunktion `writeCache_AB` legt den gegebenen Zustand `state` und Spieler `player` mit samt der errechneten Werte `value`, `alpha` und `beta` im Cache ab. Ist zusätzlich der Parameter `symmetry` auf `True` gesetzt, werden die errechneten Werte ebenfalls für alle symmetrischen Zustände gespeichert.

In [ ]:
def writeCache_AB(state, player, value, alpha, beta, symmetry=True):
    global Cache
    
    # If symmetry is enabled, find all symmetic states and save them too
    if symmetry:
        for symmetricState in findSymmetries(state):
            Cache[(symmetricState, player)] = (value, alpha, beta)
    else:
        Cache[(state, player)] = (value, alpha, beta)

Die Exception `IterativeMaxCountException` wird aufgerufen, wenn ein Fehler aufgrund des Erreichens der maximal zu betrachtenden Zustände bei der iterativen Tiefensuche erzeugt werden soll.

In [ ]:
class IterativeMaxCountException(Exception):
    def __init__(self, maxCount):
        super().__init__(f"Reached max count ({maxCount}) of iterative deepeing")    

Die Klasse `IterativeCounter` implementiert zum einen die Werte `maxCount` und `value` und zum anderen die Methoden `increment` und `check`.
Die Methode `increment` erhöht den `value` im eins, in der Methoden `check` wird zusätzlich noch überprüft, ob der neu erhöhte `value` den `maxCount` überschreitet und erzeugt eine `IterativeMaxCountException`.

Die Implementierung als Klasse ermöglicht es, eine Referenz eines Objekts der Klasse an die Funktionen `value_AB` und `alphaBeta` zu übergeben. Dadurch wird effektiv in allen Teilbäumen und allen Rekursionstiefen das gleiche Objekt aufgerufen und somit der `value` korrekt gezählt.

In [ ]:
class IterativeCounter:
    def __init__(self, maxCount = 500, value = 0):
        self.maxCount = maxCount
        self.value = value
        
    def increment(self):
        self.value += 1
        
    def check(self):
        self.increment()
        if (self.value > self.maxCount):
            raise IterativeMaxCountException(self.maxCount)

Die Funktion `orderMove` sortiert eine Liste von nächsten Zuständen `ns` für einen Spieler `p` entsprechend der für diese Zustände erreichten Werte aus der vorherigen Iteration `OldCache`. So soll erreicht werden, dass vielversprechende Zustände früher betrachtet werden sollen und sich somit Teilbäume eventuell früher abschneiden lassen.

In [ ]:
def orderMove(ns, p):
    global OldCache
    return sorted(ns, key = lambda s: OldCache.get((s, p), (0, -1, 1))[0], reverse=True)

Die `value_AB()` Funktion ist ein Wrapper für die eigentliche Implementierung des α-β-Pruning in der Funktion `alphaBeta()`. Dieser Wrapper stellt Caching der Ergebnisse zur Verfügung, bzw. berechnet diese neu, falls der Cache keine oder invalide Ergenisse beinhaltet.

Ein Ergebnis aus dem Cache ist valide, solange das Intervall `alpha` und `beta` aus den Parametern innerhalb des im Cache verwendeten Intervalls `a` und `b` liegt. Also das Intervall des Caches muss genereller sein, als das Intervall aus den Paramtern.

In [ ]:
Cache = {}
OldCache = {}

def value_AB(s, p, alpha=-1, beta=1, limit=3, symmetry=True, counter = None):
    global Cache
    if (s, p) in Cache:
        (val, a, b) = Cache[(s, p)]
        if a <= alpha and beta <= b:
            return val
        else:
            alpha = min(alpha, a)
            beta  = max(beta , b)
            val   = alphaBeta(s, p, alpha, beta, limit=limit, symmetry=symmetry, counter=counter)
            writeCache_AB(s, p, val, alpha, beta, symmetry=symmetry)
            return val
    else:
        val = alphaBeta(s, p, alpha, beta, limit=limit, symmetry=symmetry, counter=counter)
        writeCache_AB(s, p, val, alpha, beta, symmetry=symmetry)
        return val

Die Funktion `alphaBeta()` beinhaltet nun die eigentliche Implementierung des α-β-Pruning.

* Wie zuvor beim Minimax Algorithmus, wird der `utility` Wert zurückgegeben, falls das Spiel in dem State `s` beendet (`finished`) ist.
* Ebenfalls äquivalent wird der `heuristic` Wert verwendet, sobald das Rekursionslimit (`limit`) erreicht wird.
* Außerdem wird der Zähler für die iterative Tiefensuche erhöht, sofern ein Objekt der Klasse `IterativeCounter` übergeben wird.
* Der eigentliche α-β-Pruning Alogrithmus errechnet rekursiv mit Hilfe des Caches (`value_AB()`) den Wert eines Zuges. Hierbei wird der erste Wert der nächsten States verwendet, der größer oder gleich der oberen Grenze `beta` ist.

In [ ]:
def alphaBeta(s, p, alpha, beta, limit=3, symmetry=True, counter = None):
    if counter:
        counter.check()
    if limit == 0:
        return heuristic(s, p)

    states = nextStates(s, p)
    if finished(s, p, ns=states):
        return utility(s, p, ns=states)

    val = alpha
    for ns in orderMove(states, p):
        val = max(val, -value_AB(ns, opponent(p), -beta, -alpha, limit=limit-1, symmetry=symmetry, counter=counter))
        if val >= beta:
            return val
        alpha = max(val, alpha)
    return val

`bestMove_AB()` wählt mit Hilfe des α-β-Pruning Algorithmus den besten State aus allen möglichen nächsten States aus. Hierzu werden die Werte aller States errechnet und der State mit dem höchsten Wert ausgewählt. Dies wird solange ausgeführt, bis der maximale Wert der iterativen Tiefensuche erreicht ist und einen Fehler erzeugt.

In [ ]:
def bestMove_AB(s, p, symmetry=True, maxCount=25000):
    # Clear cache
    global Cache
    global OldCache
    Cache = {}
    OldCache = {}
    
    counter = IterativeCounter(maxCount=maxCount)
    limit = 1

    ns = nextStates(s, p)
    moves = [(0, s) for s in ns]

    while True:
        try:
            moves = [
                (-value_AB(s, opponent(p), limit=limit, symmetry=symmetry, counter=counter), s)
                for s in ns
            ]
            OldCache = Cache
            Cache = {}
            limit += 1
        except IterativeMaxCountException:
            print(f"Max count reached at {limit}")
            break
    
    return max(moves, key=lambda m: m[0])